In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"kapon00","key":"3cb8c2c5feb62cc2774af0f2f6afe05e"}'}

In [2]:
#kaggle api
!pip install -q kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#this lets avoid a warning on Kaggle tool startup
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!mkdir dog_dataset
%cd dog_dataset

mkdir: cannot create directory ‘dog_dataset’: File exists
/content/dog_dataset


In [5]:
#searching for dataset
!kaggle datasets list -s dogbreedidfromcomp


ref                                title                    size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------  ----------------------  -----  -------------------  -------------  ---------  ---------------  
catherinehorng/dogbreedidfromcomp  dog-breed-id-from-comp  691MB  2020-06-26 03:09:05           7600         10  0.1764706        


In [6]:
#downloading dataset
!kaggle datasets download catherinehorng/dogbreedidfromcomp
%cd ..

Dataset URL: https://www.kaggle.com/datasets/catherinehorng/dogbreedidfromcomp
License(s): unknown
 99% 685M/691M [00:09<00:00, 81.8MB/s]
100% 691M/691M [00:09<00:00, 75.6MB/s]
/content


In [7]:
#managing files
!unzip dog_dataset/dogbreedidfromcomp.zip -d dog_dataset
!rm dog_dataset/dogbreedidfromcomp.zip
!rm dog_dataset/sample_submission.csv

Archive:  dog_dataset/dogbreedidfromcomp.zip
replace dog_dataset/labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
  inflating: dog_dataset/sample_submission.csv  
replace dog_dataset/test/000621fb3cbb32d8935728e48679680e.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: rm: cannot remove 'dog_dataset/dogbreedidfromcomp.zip': No such file or directory
rm: cannot remove 'dog_dataset/sample_submission.csv': No such file or directory


In [8]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm #for better process visualization
from keras.preprocessing import image
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam


In [9]:
labels_all = pd.read_csv('dog_dataset/labels.csv') #reading the data
labels_all.shape
labels_all.head()


,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [10]:
breeds_all = labels_all['breed']
breeds_count = breeds_all.value_counts()
breeds_count.head()

,count
breed,
scottish_deerhound,126
maltese_dog,117
afghan_hound,116
entlebucher,115
bernese_mountain_dog,114


In [11]:
#limitation of breeds for better performance (computation power limits)
CLASS_NAMES = ['scottish_deerhound','maltese_dog','bernese_mountain_dog']
labels = labels_all[labels_all['breed'].isin(CLASS_NAMES)]
labels = labels.reset_index()
labels.head()


,index,id,breed
0,9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
1,12,00693b8bc2470375cc744a6391d397ec,maltese_dog
2,79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound
3,90,022b34fd8734b39995a9f38a4f3e7b6b,maltese_dog
4,118,02d54f0dfb40038765e838459ae8c956,bernese_mountain_dog


In [12]:
#matrix of zeros
X_data = np.zeros((len(labels), 224,224,3), dtype = 'float32')
#one hot encoding
y_data = label_binarize(labels['breed'], classes = CLASS_NAMES)

#Reading and converting image to an array and normalizing it
for i in tqdm(range(len(labels))):
  img = image.load_img('dog_dataset/train/%s.jpg'% labels['id'][i], target_size = (224,224))
  img = image.img_to_array(img)
  x = np.expand_dims(img.copy(), axis = 0)
  X_data[i] = x / 255.0

  print ('\nTrain Images shape: ', X_data.shape, ' size: {:,}'.format(X_data.size))
  print ('\nEncoded output shape: ', y_data.shape, ' size: {:,}'.format(y_data.size))

 20%|█▉        | 70/357 [00:00<00:00, 340.08it/s]


Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,

 43%|████▎     | 154/357 [00:00<00:00, 393.25it/s]


Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,

 67%|██████▋   | 239/357 [00:00<00:00, 410.86it/s]


Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,

 92%|█████████▏| 328/357 [00:00<00:00, 426.07it/s]


Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,

100%|██████████| 357/357 [00:00<00:00, 401.02it/s]


Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,496

Encoded output shape:  (357, 3)  size: 1,071

Train Images shape:  (357, 224, 224, 3)  size: 53,738,